# Color Processing

This notebook presents some practical exercises on color processing.

## Dependencies

Please make sure the following packages are installed.

```bash
sudo pip3 install pandas
```

## Common Imports

Make sure you run this kernel first, to import once every dependency.

In [ ]:
import pandas as pd
import numpy as np
import cv2
from matplotlib import pyplot as plt
from ipywidgets import interact, interactive
import ipywidgets as widgets

## RGB Images in OpenCV

RGB images in OpenCV are handled as a 3 dimensional arrays as $w\times h \times c$ where $w$,$h$ and $c$ are the width, height and channels respectively. For the specific RGB case, $c$ is 3.

Let's experiment with these channels.

In [ ]:
img = cv2.imread('landscape.jpeg')

# Double check the image dimensions
h,w,c = img.shape
print(w,h,c)

# Display the original image
plt.imshow(cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
plt.figure()

B,G,R=0,1,2
tocopy=B

channel = np.uint8(np.zeros((h,w,c)))

# Naive implementation, dont do this!!!
for row in range(h):
    for col in range(w):
        channel[row,col,tocopy] = img[row,col,tocopy]

# Slice approach, much more efficient
#channel[:,:,tocopy] = img[:,:,tocopy]

plt.imshow(cv2.cvtColor(channel, cv2.COLOR_RGB2BGR))

### Practice

Lets build an image that is formed by 1/3rd of the red channel, 1/3rd of the blue channel, and 1/3rd of the green channel of the original image.

```
         .---------.
         |   RED   |
         |---------|
         |  GREEN  |
         |---------|
         |  BLUE   |
         '---------'
```

In [ ]:
img = cv2.imread('landscape.jpeg')

# Double check the image dimensions
h,w,c = img.shape
print(w,h,c)

# Write here the built image
meld = np.uint8(np.zeros((h,w,c)))

# Display the original image
plt.imshow(cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
plt.figure()

# Display the output image
plt.imshow(cv2.cvtColor(meld, cv2.COLOR_RGB2BGR))

## Pseudocolor

The following exercise allows you to add pseudocolor to a a real-life thermal camera output using LUT (Look Up Tables). Use OpenCV and Pandas to try out different thermal palettes.

1. Read `palettes.csv` using Pandas
2. Choose one of the palettes
3. Build an RGB LUT

The palet will contain a single integer in hex format with `0xAABBGGRR`. For example:
```python
color = 0xff0c10fa
A = 255 # 0xff
B = 12 # 0x0c
G = 16 # 0x10
R = 250 # 0xfa
```

4. Load `thermal.avi` and read frame by frame
5. Iterate through all pixels using its value to index the LUT
6. Build and display an artificially colored image. (Use OpenCV's imshow to allow animations)

In [ ]:
# How to read all the palettes
data = pd.read_csv('palettes.csv')

# How to read a single pallete
palette = data['ironbow']

LUT = np.zeros((256,3), dtype='uint8')

for i in range(256):
    color = int(palette[i],0)

    A = (color & 0xff000000) >> 24
    B = (color & 0x00ff0000) >> 16
    G = (color & 0x0000ff00) >> 8
    R = (color & 0x000000ff)
    LUT[i] = [B, G, R]

cap = cv2.VideoCapture('thermal_small.avi')
img = np.zeros((240,320,3), dtype='uint8')

tune = 0.5

print ("Starting")
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print ("End of video")
        break
        
    for row in range(frame.shape[0]):
        for col in range(frame.shape[1]):
            index = int(frame[row,col,1])
            img[row,col] = LUT[index]
    
    # Display the resulting frame using matplotlib
    #plt.imshow(img)
    #plt.show()
    
    # Display the resulting frame using opencv
    cv2.imshow('frame', np.uint8(img*tune))
    if cv2.waitKey(33) == 'q':
        break


### Practice

Can you think of a more efficient way to achieve this using slice processing?

In [ ]:
# How to read all the palettes
data = pd.read_csv('palettes.csv')

# How to read a single pallete
palette = data['ironbow']

LUT = np.zeros((256,3), dtype='uint8')

for i in range(256):
    color = int(palette[i],0)

    A = (color & 0xff000000) >> 24
    B = (color & 0x00ff0000) >> 16
    G = (color & 0x0000ff00) >> 8
    R = (color & 0x000000ff)
    LUT[i] = [B, G, R]

cap = cv2.VideoCapture('thermal_small.avi')
img = np.zeros((240,320,3), dtype='uint8')

tune = 0.5

print ("Starting")
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print ("End of video")
        break

    # PROCESS HERE
    # PROCESS HERE
    # PROCESS HERE
    
    # Display the resulting frame using matplotlib
    #plt.imshow(img)
    #plt.show()
    
    # Display the resulting frame using opencv
    cv2.imshow('frame', np.uint8(img*tune))
    if cv2.waitKey(33) == 'q':
        break

## Color Spaces

You can think of color spaces as different coordinate axis that span spaces where colors can be defined. In digital medias it is common to represent colors in device-dependent color-spaces such as `RGB` or `Y'C'bC'r`.

OpenCV provides severl built-in color-space-transformation utilities. See [the official documentation](https://docs.opencv.org/3.1.0/de/d25/imgproc_color_conversions.html) and [some extra formats](https://docs.opencv.org/3.1.0/d7/d1b/group__imgproc__misc.html).

### RGB to YCbCr (UYVY)

Very often you will require fine-grained control over your images color format. In this exercise you will manually convert an `RGB` image to `YCbCr` in the [UYVY format](https://www.fourcc.org/pixel-format/yuv-uyvy/). Note that OpenCV handles the format in a slightly different format (two channels).


In [ ]:
rgb = cv2.imread('lenna.png')

rows, cols, _ = rgb.shape
uyvy = np.zeros((rows, cols, 2), dtype='uint8')
print(uyvy.shape)

for row in range(0, rows):
    for col in range(0, cols, 2):
        B1, G1, R1 = rgb[row, col]
        B2, G2, R2 = rgb[row, col + 1]
        
        y1 = R1*0.299 + G1*0.587 + B1*0.114
        y2 = R2*0.299 + G2*0.587 + B2*0.114
        u = (R1 - y1)*0.713 + 128
        v = (B1 - y1)*0.564 + 128
        
        uyvy[row, col + 0, 0] = u
        uyvy[row, col + 1, 0] = v
        uyvy[row, col + 0, 1] = y1
        uyvy[row, col + 1, 1] = y2

# Convert Back to RGB to display it
plt.imshow(cv2.cvtColor (uyvy, cv2.COLOR_YUV2BGR_UYVY))

### RGB to YCbCr (YUY2)

Repeat the exercise above but for the [YUY2 Format](https://www.fourcc.org/pixel-format/yuv-yuy2/)

In [ ]:
# Try it yourself

### RGB to YCbCr (I420)

Repeat the exercise above but for the [I420 Format](https://www.fourcc.org/pixel-format/yuv-i420/)

In [ ]:
# Try it yourself

## Histogram Equalization
### Grayscale Historgram Equalization

To start of, load `landscape.jpeg` in grayscale and equalize its histogram using [cv2.equalizeHistogram](https://docs.opencv.org/3.1.0/d6/dc7/group__imgproc__hist.html#ga7e54091f0c937d49bf84152a16f76d6e)

Additional references:
* [Histogram Computation Example](https://docs.opencv.org/3.1.0/d1/db7/tutorial_py_histogram_begins.html)
* [Histogram Equalization Example](https://docs.opencv.org/3.1.0/d4/d1b/tutorial_histogram_equalization.html)


In [ ]:
img = cv2.imread('landscape.jpeg', cv2.IMREAD_GRAYSCALE)
hist = cv2.calcHist([img],[0],None,[256],[0,256])

plt.imshow(img, cmap='gray')
plt.title('Original Image')
plt.figure()

plt.plot (hist)
plt.title('Histogram')
plt.figure()

# Equalize and display new image and histogram
img2 = cv2.equalizeHist(img)
hist2 = cv2.calcHist([img2],[0],None,[256],[0,256])

plt.imshow(img2, cmap='gray')
plt.title('Equalized Image')
plt.figure()

plt.plot (hist2)
plt.title('Equalized Histogram')
plt.figure()

### Independent Channel Equalization

Now load the image in RGB format, and experiment by equalizing each channel independently, and combinations of them.

In [ ]:
@interact(R=True, G=False, B=False)
def equalize(R, G, B):
    img = cv2.imread('landscape.jpeg')
    blue = img[:,:,0]
    green = img[:,:,1]
    red = img[:,:,2]

    if B == True:
        blue = cv2.equalizeHist(blue)
    if G == True:
        green = cv2.equalizeHist(green)
    if R == True:
        red = cv2.equalizeHist(red)
        
    img[:,:,0] = blue
    img[:,:,1] = green
    img[:,:,2] = red
    
    # Practice! Display each the color histograms
    
    # This is required since MatPlotLib and OpenCV don't agree in RGB
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    
    plt.imshow(img)

### Complete Color Equalization

Finally, load the image in RGB format, convert to YCbCr and equalize the luminance instead.

In [ ]:
@interact(Y=True, U=False, V=False)
def equalize(Y, U, V):
    img = cv2.imread('landscape.jpeg')
    
    yuv = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)

    if Y:
        yuv[:,:,0] = cv2.equalizeHist(yuv[:,:,0])
    if U:
        yuv[:,:,1] = cv2.equalizeHist(yuv[:,:,1])
    if V:
        yuv[:,:,2] = cv2.equalizeHist(yuv[:,:,2])

    img2 = cv2.cvtColor(yuv, cv2.COLOR_YUV2RGB)

    plt.imshow(cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    plt.figure()
    plt.imshow(cv2.cvtColor(img2, cv2.COLOR_RGB2BGR))

## Color Probability Maps

Now let's implement a naive version of the color probability maps algorithm. This implementation will be based on the original paper: [Statistical Color Models with Applications on Skin Detection](https://www.hpl.hp.com/techreports/Compaq-DEC/CRL-98-11.pdf). The data set was taken from the [UCI Skin Detection Data Set](https://archive.ics.uci.edu/ml/datasets/skin+segmentation)

In [ ]:
data = pd.read_csv('Skin_NonSkin.txt', sep='\t', names=['B', 'G', 'R', 'Class'])
print(data)

is_skin = data['Class'] == 1
is_not_skin = data['Class'] == 2

skin = data[is_skin]
non_skin = data[is_not_skin]

print(skin.shape)
print(non_skin.shape)

In [ ]:
number_of_bins = 32
bins=255/number_of_bins

H = {}
for index, row in data.iterrows():
    pos = (int(row['B']/bins), int(row['G']/bins), int(row['R']/bins))
    if not H.get(pos):
        H[pos] = 0
    H[pos] += 1

In [ ]:
H_skin = {}
for index, row in skin.iterrows():
    pos = (int(row['B']/bins), int(row['G']/bins), int(row['R']/bins))
    if not H_skin.get(pos):
        H_skin[pos] = 0
    H_skin[pos] += 1

In [ ]:
H_non_skin = {}
for index, row in non_skin.iterrows():
    pos = (int(row['B']/bins), int(row['G']/bins), int(row['R']/bins))
    if not H_non_skin.get(pos):
        H_non_skin[pos] = 0
    H_non_skin[pos] += 1

In [ ]:
Ts = len(skin)
Tn = len(non_skin)
T = len(data)
P_skin = Ts/(Ts + Tn)
P_non_skin = 1 - P_skin

img = cv2.imread('gesture_accept.png')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

rows, cols, _ = img.shape

spm = np.zeros((rows,cols))

for row in range(0,rows):
    for col in range(0,cols):
        B,G,R = img[row,col]
        
        B = int(B/bins)
        G = int(G/bins)
        R = int(R/bins)
        
        has_value = H_skin.get((B,G,R))
        P_rgb_skin = 0 if not has_value else has_value
        P_rgb_skin /= Ts
        
        has_value = H_non_skin.get((B,G,R))
        P_rgb_non_skin = 0 if not has_value else has_value
        P_rgb_non_skin /= Tn
        
        has_value = H.get((B,G,R))
        P_rgb = 0 if not has_value else has_value
        P_rgb /= T

        spm[row,col] = (P_rgb_skin*P_skin)/(P_rgb_skin*P_skin + P_rgb_non_skin*P_non_skin + 1e-6)

plt.figure()
plt.imshow(spm, cmap='gray')
print(np.max(spm))

In [ ]:
print (np.max(spm))

In [ ]:
cam = cv2.VideoCapture(2)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)

while True:
    ret, img = cam.read()
    rows, cols, _ = img.shape

    spm = np.zeros((rows,cols))

    for row in range(0,rows):
        for col in range(0,cols):
            B,G,R = img[row,col]
        
            B = int(B/bins)
            G = int(G/bins)
            R = int(R/bins)
        
            has_value = H_skin.get((B,G,R))
            P_rgb_skin = 0 if not has_value else has_value
            P_rgb_skin /= Ts
        
            has_value = H_non_skin.get((B,G,R))
            P_rgb_non_skin = 0 if not has_value else has_value
            P_rgb_non_skin /= Tn
        
            has_value = H.get((B,G,R))
            P_rgb = 0 if not has_value else has_value
            P_rgb /= T

            spm[row,col] = (P_rgb_skin*P_skin)/(P_rgb_skin*P_skin + P_rgb_non_skin*P_non_skin + 1e-6)

    cv2.imshow('Probability Map', spm)
    cv2.imshow('Original', img)
    
    if (cv2.waitKey(1) & 0xFF) == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()
print(min(np.reshape(spm, spm.size,1)))
